In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn
from topic_model import TopicModel

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '0', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/tglm_vae_tmp0', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 500, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.01, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 10, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_test_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

# build language model 

## feed dict

In [8]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow], name='bow')
t_variables['input_token_idxs'] = tf.placeholder(tf.int32, [None, None], name='input_token_idxs')
t_variables['dec_input_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_input_idxs')
t_variables['dec_target_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_target_idxs')
t_variables['batch_l'] = tf.placeholder(tf.int32, name='batch_l')
t_variables['doc_l'] = tf.placeholder(tf.int32, [None], name='doc_l')
t_variables['sent_l'] = tf.placeholder(tf.int32, [None], name='sent_l')
t_variables['keep_prob'] = tf.placeholder(tf.float32, name='keep_prob')

In [9]:
def get_feed_dict(batch, mode='train', assertion=False):
    def token_dropout(sent_idxs):
        sent_idxs_dropout = np.asarray(sent_idxs)
        sent_idxs_dropout[np.random.rand(len(sent_idxs)) > config.word_keep_prob] = config.UNK_IDX
        return list(sent_idxs_dropout)

    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    
    doc_l = np.array([len(instance.token_idxs) for instance in batch])
    
    feed_input_token_idxs_list = [sent_idxs for instance in batch for sent_idxs in instance.token_idxs]
    feed_dec_input_idxs_list = [[config.BOS_IDX] + token_dropout(sent_idxs) for sent_idxs in feed_input_token_idxs_list]
    feed_dec_target_idxs_list = [sent_idxs + [config.EOS_IDX]  for sent_idxs in feed_input_token_idxs_list]
        
    sent_l = np.array([len(sent_idxs) for sent_idxs in feed_input_token_idxs_list], np.int32)
    batch_l = len(sent_l)
    
    feed_input_token_idxs = pad_sequences(feed_input_token_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_input_idxs = pad_sequences(feed_dec_input_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_target_idxs = pad_sequences(feed_dec_target_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    
    if assertion:
        index = 0
        for instance in batch:
            for line_idxs in instance.token_idxs:
                assert feed_input_token_idxs_list[index] == line_idxs
                index += 1
        assert feed_input_token_idxs.shape[1] == np.max(sent_l)
        assert feed_dec_input_idxs.shape[1] == np.max(sent_l) + 1
        assert feed_dec_target_idxs.shape[1] == np.max(sent_l) + 1
    
    keep_prob = config.keep_prob if mode == 'train' else 1.0

    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['batch_l']: batch_l, t_variables['doc_l']: doc_l, t_variables['sent_l']: sent_l, 
                t_variables['input_token_idxs']: feed_input_token_idxs, t_variables['dec_input_idxs']: feed_dec_input_idxs, t_variables['dec_target_idxs']: feed_dec_target_idxs, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [10]:
def debug_shape(variables):
    sample_batch = dev_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    
# sent_loss_kl_categ_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, tf_log(prob_topic_infer/prob_topic_sents)), 1))
# debug_value([sent_loss_kl_categ, sent_loss_kl_categ_tmp])
# sent_loss_kl_gauss_tmp = 0.5 * tf.reduce_sum(tf.exp(logvars_topic_infer-logvars_topic) + tf.square(means_topic - means_topic_infer) / tf.exp(logvars_topic) - 1 + (logvars_topic - logvars_topic_infer), -1)
# sent_loss_kl_gmm_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss_tmp), -1))
# debug_value([sent_loss_kl_gmm_tmp, sent_loss_kl_gmm])    

## topic model

In [11]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors

    prob_topic = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic')(latents_bow) # inference of topic probabilities

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    topic_embeddings = tf.get_variable('topic_emb', [config.n_topic, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics

    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

    # prior of each gaussian distribution (computed for each topic)
    hidden_topic = tf.layers.Dense(units=config.dim_hidden_topic, activation=tf.nn.relu, name='hidden_topic')(topic_bow)
    means_topic = tf.layers.Dense(units=config.dim_latent, name='mean_topic')(hidden_topic)
    logvars_topic = tf.layers.Dense(units=config.dim_latent, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_topic)
    sigma_topic = tf.exp(0.5 * logvars_topic)
    gauss_topic = tfd.Normal(loc=means_topic, scale=sigma_topic)    
    
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(config.n_topic)))
# topic_angles = tf.acos(topic_dots)
# topic_angles_mean = tf.reduce_mean(topic_angles)
# topic_angles_vars = tf.reduce_mean(tf.square(topic_angles - topic_angles_mean))
# topic_loss_reg = tf.exp(topic_angles_vars - topic_angles_mean)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

## encoder

In [12]:
# input
input_token_idxs = t_variables['input_token_idxs']
batch_l = t_variables['batch_l']
sent_l = t_variables['sent_l']
max_sent_l = tf.reduce_max(sent_l)

with tf.variable_scope('sent/enc', reuse=False):
    # get word embedding
    enc_input = tf.nn.embedding_lookup(embeddings, input_token_idxs)

    # get sentence embedding
    _, enc_state = dynamic_bi_rnn(enc_input, sent_l, config.dim_hidden, t_variables['keep_prob'])

    # TODO House Holder flow
    hidden_topic_infer =  tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='hidden_topic_infer')(enc_state)
    prob_topic_infer = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic_infer')(hidden_topic_infer)

    w_mean_topic_infer = tf.get_variable('mean_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32)
    b_mean_topic_infer = tf.get_variable('mean_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32)
    means_topic_infer = tf.tensordot(enc_state, w_mean_topic_infer, axes=[[1], [1]]) + b_mean_topic_infer
    
    w_logvar_topic_infer = tf.get_variable('logvar_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    b_logvar_topic_infer = tf.get_variable('logvar_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    logvars_topic_infer = tf.tensordot(enc_state, w_logvar_topic_infer, axes=[[1], [1]]) + b_logvar_topic_infer
    sigma_topic_infer = tf.exp(0.5 * logvars_topic_infer)
    gauss_topic_infer = tfd.Normal(loc=means_topic_infer, scale=sigma_topic_infer)
    
    # latent vectors from each gaussian dist.
    latents_topic_infer = sample_latents(means_topic_infer, logvars_topic_infer) 
    # latent vector from gaussian mixture
    latents_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), latents_topic_infer, transpose_a=True)
    
    # for beam search
    means_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), means_topic_infer, transpose_a=True)    

## decoder

In [13]:
# prepare for decoding
dec_sent_l = tf.add(sent_l, 1)
dec_input_idxs = t_variables['dec_input_idxs']
dec_input = tf.nn.embedding_lookup(embeddings, dec_input_idxs)

dec_latents_input = tf.tile(latents_input, [1, tf.shape(dec_input)[1], 1])
dec_concat_input = tf.concat([dec_input, dec_latents_input], -1)

# decode for training
with tf.variable_scope('sent/dec/rnn', initializer=tf.contrib.layers.xavier_initializer(), dtype = tf.float32, reuse=False):
    dec_cell = tf.contrib.rnn.GRUCell(config.dim_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob = t_variables['keep_prob'])

    dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(latents_input, 1))
    
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_concat_input, sequence_length=dec_sent_l)

    train_decoder = tf.contrib.seq2seq.BasicDecoder(
        cell=dec_cell,
        helper=helper,
        initial_state=dec_initial_state)

    dec_outputs, _, output_sent_l = tf.contrib.seq2seq.dynamic_decode(train_decoder)
    
    output_layer = tf.layers.Dense(config.n_vocab, use_bias=False, name='out')
    output_logits = output_layer(dec_outputs.rnn_output)
    
    output_token_idxs = tf.argmax(output_logits, 2)

In [14]:
start_tokens = tf.fill([batch_l], config.BOS_IDX)
end_token = config.EOS_IDX

with tf.variable_scope('sent/dec/rnn', reuse=True):
    infer_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(means_input, 1))
    beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(infer_dec_initial_state, multiplier=config.beam_width)
    beam_latents_input = tf.contrib.seq2seq.tile_batch(tf.squeeze(means_input, 1), multiplier=config.beam_width) # added
    
    beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=beam_latents_input)

    beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        beam_decoder,
        maximum_iterations = config.maximum_iterations)

    beam_output_token_idxs = beam_dec_outputs.predicted_ids[:, :, 0]

In [15]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    inter_means_input = tf.placeholder(tf.float32, [None, config.dim_latent])
    
    inter_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(inter_means_input)
    inter_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(inter_dec_initial_state, multiplier=config.beam_width)
    inter_beam_latents_input = tf.contrib.seq2seq.tile_batch(inter_means_input, multiplier=config.beam_width) # added
    
    inter_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=inter_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=inter_beam_latents_input)

    inter_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        inter_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    inter_beam_output_token_idxs = inter_beam_dec_outputs.predicted_ids[:, :, 0]

In [16]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    topic_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic)
    topic_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(topic_dec_initial_state, multiplier=config.beam_width)
    topic_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic, multiplier=config.beam_width) # added
    
    topic_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=topic_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=topic_beam_latents_input)

    topic_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        topic_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    topic_beam_output_token_idxs = topic_beam_dec_outputs.predicted_ids[:, :, 0]

In [17]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    means_topic_summary = tf.reduce_mean(means_topic_infer, 0)
    
    summary_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic_summary)
    summary_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(summary_dec_initial_state, multiplier=config.beam_width)
    summary_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic_summary, multiplier=config.beam_width) # added
    
    summary_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=summary_beam_dec_initial_state,
        beam_width=config.beam_width,
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=summary_beam_latents_input)

    summary_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        summary_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    summary_beam_output_token_idxs = summary_beam_dec_outputs.predicted_ids[:, :, 0]

## language modeling cost

In [18]:
# target and mask
dec_target_idxs = t_variables['dec_target_idxs']
dec_mask_tokens = tf.sequence_mask(dec_sent_l, maxlen=max_sent_l+1, dtype=tf.float32)

# nll for each token (averaged over batch & sentence)
sent_loss_recon = tf.contrib.seq2seq.sequence_loss(output_logits, dec_target_idxs, dec_mask_tokens)

In [19]:
doc_l = t_variables['doc_l']
mask_sents = tf.sequence_mask(doc_l)
mask_sents_flatten = tf.reshape(mask_sents, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1]])

prob_topic_tiled = tf.tile(tf.expand_dims(prob_topic, 1), [1, tf.shape(mask_sents)[1], 1])
prob_topic_flatten = tf.reshape(prob_topic_tiled, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1], config.n_topic])
prob_topic_sents = tf.boolean_mask(prob_topic_flatten, mask_sents_flatten)

In [20]:
# inferred mixture probabilities (computed for each sentence)
categ_topic_infer = tfd.Categorical(probs=prob_topic_infer)

# prior of mixture probabilities (computed for each document, tiled for each sentence)
categ_topic = tfd.Categorical(probs=prob_topic_sents)

sent_loss_kl_categ = tf.reduce_mean(tfd.kl_divergence(categ_topic_infer, categ_topic))

# inference of each gaussian gaussribution (computed for each sentence)

sent_loss_kl_gauss = tf.reduce_sum(tfd.kl_divergence(gauss_topic_infer, gauss_topic), -1)
sent_loss_kl_gmm = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss), -1))

sent_loss_kl = sent_loss_kl_categ + sent_loss_kl_gmm

## optimizer

In [21]:
global_step = tf.Variable(0, name='global_step',trainable=False)
tau = tf.cast(tf.divide(tf.mod(global_step, tf.constant(config.cycle_steps)), tf.constant(config.cycle_steps)), dtype=tf.float32)
beta = tf.minimum(1., tau/config.r_cycle)

sent_loss = sent_loss_recon + beta * sent_loss_kl

topic_loss = topic_loss_recon + topic_loss_kl + config.reg * topic_loss_reg
loss = topic_loss + sent_loss

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)
    
grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]

opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

/home/m-isonuma/.pyenv/versions/anaconda2-5.3.0/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# run model 

In [22]:
def idxs_to_sents(token_idxs, config, idx_to_word):
    sents = []
    for sent_idxs in token_idxs:
        tokens = []
        for idx in sent_idxs:
            if idx == config.EOS_IDX: break
            tokens.append(idx_to_word[idx])
        sent = ' '.join(tokens)
        sents.append(sent)
    return sents

In [23]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, sent_loss_recon_mean, sent_loss_kl_mean, ppl_mean

def get_all_losses(sess, batches):
    losses = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch = \
        sess.run([loss, topic_loss_recon, topic_loss_kl, sent_loss_recon, sent_loss_kl], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
    print('LOSS %.2f | TM NLL: %.2f, KL: %.4f | LM NLL: %.2f, KL: %.4f' %  np.mean(losses, 0))

In [24]:
def print_sample(sample_batch):
    feed_dict = get_feed_dict(sample_batch)
    pred_token_idxs = sess.run(output_token_idxs, feed_dict = feed_dict)
    true_token_idxs = [sent_idxs for instance in sample_batch for sent_idxs in instance.token_idxs]
    
    assert len(pred_token_idxs) == len(true_token_idxs)
    
    pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)
    true_sents = idxs_to_sents(true_token_idxs, config, idx_to_word)
    
    for i, (true_sent, pred_sent) in enumerate(zip(true_sents, pred_sents)):        
        print(i, 'TRUE: %s' % true_sent)
        print(i, 'PRED: %s' % pred_sent)

def print_topic_sample():
    pred_topics_freq_bow_indices, pred_topic_token_idxs = sess.run([topics_freq_bow_indices, topic_beam_output_token_idxs], 
                                                                                                           feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})
    pred_topic_sents = idxs_to_sents(pred_topic_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Topic Samples-----------')
    for i, (topic_freq_bow_idxs, pred_topic_sent) in enumerate(zip(topics_freq_bow_idxs, pred_topic_sents)):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' SENTENCE:', pred_topic_sent)
        
def print_summary(test_batch):
    feed_dict = get_feed_dict(test_batch)
    feed_dict[t_variables['batch_l']] = config.n_topic
    feed_dict[t_variables['keep_prob']] = 1.
    pred_topics_freq_bow_indices, pred_summary_token_idxs = sess.run([topics_freq_bow_indices, summary_beam_output_token_idxs], feed_dict=feed_dict)
    pred_summary_sents = idxs_to_sents(pred_summary_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Output sentences for each topic-----------')
    print('Item idx:', test_batch[0].item_idx)
    for i, (topic_freq_bow_idxs, pred_summary_sent) in enumerate(zip(topics_freq_bow_idxs, pred_summary_sents)):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' SENTENCE:', pred_summary_sent)
        
    print('-----------Summaries-----------')
    for i, summary in enumerate(test_batch[0].summaries):
        print('SUMMARY %i :'%i, '\n', summary)

In [25]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','LM','','VALID:','TM','','','','LM','', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','NLL','KL','LOSS','PPL','NLL','KL','REG','NLL','KL', 'Beta']]))))

In [26]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, sent_loss_kl_categ_batch, sent_loss_kl_gmm_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, sent_loss_kl_categ, sent_loss_kl_gmm, topic_ppls], feed_dict = feed_dict)
   
        if sent_loss_kl_batch == np.inf:
            print('Nan occured')
            ckpt = tf.train.get_checkpoint_state(config.modeldir)
            model_checkpoint_path = ckpt.all_model_checkpoint_paths[-1]
            saver.restore(sess, model_checkpoint_path)            
            break
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train, sent_loss_recon_train, sent_loss_kl_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, sent_loss_recon_dev, sent_loss_kl_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log, beta_eval = sess.run([tf.train.get_global_step(), beta])
            
            if loss_dev < loss_min:
                loss_min = loss_dev
                saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, '%.2f'%sent_loss_recon_train, '%.2f'%sent_loss_kl_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, '%.2f'%sent_loss_recon_dev, '%.2f'%sent_loss_kl_dev,  '%.3f'%beta_eval],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)

            print_summary(test_batches[1][1])
            print_sample(batch)
            
            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                        LM        VALID:  \
       Time  Ep    Ct    LOSS   PPL     NLL    KL   REG   NLL    KL    LOSS   
1        20   0     0  137.42  1034  126.92  0.47  0.90  9.13  1.46  126.64   
501      70   0   500  128.30   776  118.94  0.19  0.84  8.28  1.38  113.58   
1001     67   0  1000  124.32   660  115.97  0.15  0.80  7.31  1.19  112.11   
1501     65   0  1500  122.93   622  115.06  0.13  0.77  6.87  1.01  111.86   
2001     63   0  2000  121.81   603  114.21  0.12  0.74  6.62  0.89  111.71   
2276     41   1     0  121.55   596  114.05  0.12  0.72  6.52  0.84  111.68   
2776     65   1   500  121.01   588  113.66  0.12  0.69  6.40  0.75  111.65   
3276     65   1  1000  120.85   581  113.61  0.13  0.67  6.31  0.68  111.57   
3776     64   1  1500  120.59   576  113.44  0.13  0.64  6.24  0.63  111.39   
4276     70   1  2000  120.42   573  113.34  0.13  0.62  6.18  0.58  111.32   
4551     30   2     0  120.35   571  113.31  0.13  0.61  6.16  0.55  111.33   
5051     69   2   500  120.12   569  113.14  0.14  0.59  6.12  0.51  111.25   
5551     54   2  1000  120.07   566  113.13  0.14  0.57  6.09  0.48  111.26   
6051     64   2  1500  119.95   564  113.05  0.15  0.55  6.06  0.45  111.20   
6551     74   2  2000  119.91   563  113.05  0.15  0.53  6.04  0.43  111.11   
6826     42   3     0  119.85   562  113.00  0.16  0.53  6.03  0.41  111.08   
7326     54   3   500  119.73   560  112.91  0.16  0.51  6.01  0.39  111.08   
7826     64   3  1000  119.65   559  112.86  0.17  0.49  5.99  0.37  111.04   
8326     54   3  1500  119.61   557  112.84  0.18  0.48  5.98  0.36  111.04   
8826     82   3  2000  119.55   556  112.80  0.18  0.47  5.96  0.34  110.98   
9101     30   4     0  119.54   556  112.80  0.19  0.46  5.96  0.34  111.07   
9601     64   4   500  119.44   554  112.73  0.20  0.45  5.94  0.32  110.82   
10101    55   4  1000  119.46   553  112.76  0.20  0.44  5.93  0.31  110.91   
10601    54   4  1500  119.37   552  112.68  0.21  0.43  5.92  0.30  110.90   
11101    55   4  2000  119.33   551  112.65  0.22  0.42  5.92  0.30  110.85   
11376    40   5     0  119.30   551  112.63  0.23  0.41  5.91  0.29  110.72   
11876    63   5   500  119.24   549  112.59  0.24  0.40  5.90  0.28  110.67   
12376    63   5  1000  119.22   548  112.57  0.25  0.39  5.90  0.28  110.64   
12876    54   5  1500  119.17   547  112.53  0.26  0.38  5.89  0.27  110.76   
13376    54   5  2000  119.13   546  112.50  0.27  0.37  5.88  0.27  110.68   
...     ...  ..   ...     ...   ...     ...   ...   ...   ...   ...     ...   
100101   31  44     0  116.72   481  110.26  0.94  0.08  5.30  0.21  108.59   
100601   54  44   500  116.72   481  110.25  0.94  0.08  5.30  0.21  108.67   
101101   60  44  1000  116.71   481  110.25  0.94  0.08  5.30  0.21  108.67   
101601   61  44  1500  116.71   481  110.24  0.94  0.08  5.29  0.21  108.58   
102101   60  44  2000  116.70   481  110.24  0.94  0.08  5.29  0.21  108.66   
102376   45  45     0  116.70   481  110.24  0.94  0.08  5.29  0.21  108.31   
102876   58  45   500  116.70   481  110.24  0.94  0.08  5.29  0.21  108.40   
103376   57  45  1000  116.69   481  110.23  0.95  0.08  5.29  0.21  108.37   
103876   58  45  1500  116.68   480  110.22  0.95  0.08  5.28  0.21  108.45   
104376   58  45  2000  116.68   480  110.22  0.95  0.08  5.28  0.21  108.53   
104651   33  46     0  116.68   480  110.22  0.95  0.08  5.28  0.21  108.41   
105151   62  46   500  116.67   480  110.22  0.95  0.08  5.28  0.21  108.47   
105651   61  46  1000  116.67   480  110.22  0.95  0.08  5.28  0.21  108.58   
106151   60  46  1500  116.66   480  110.22  0.95  0.08  5.27  0.21  108.45   
106651   60  46  2000  116.66   480  110.21  0.95  0.08  5.27  0.21  108.49   
106926   36  47     0  116.65   480  110.21  0.95  0.08  5.27  0.21  108.49   
107426   59  47   500  116.65   480  110.20  0.96  0.08  5.27  0.21  108.54   
107926   59  47  1000  116.65   480  110.20  0.96  0.08  5.27  0.21  108.

-----------Output sentences for each topic-----------
Item idx: B000VB7EFW
0  BOW: ! love color ... perfect recommend perfectly cute buy ordered
0  SENTENCE: it is a great bag
1  BOW: plug rip vaio functionality squeeze floor searching sewn abuse local
1  SENTENCE: it is a great bag
2  BOW: cover color bottom keyboard mac pro hard top scratches apple
2  SENTENCE: it is a great bag
3  BOW: broke neoprene thin foam loose started chromebook month returned close
3  SENTENCE: it is a great bag
4  BOW: bought & ; price quality nice 'm time perfectly buy
4  SENTENCE: it is a great bag
5  BOW: inside made - back zipper big inch put padding 've
5  SENTENCE: it is a great bag
6  BOW: sleeve protection material zipper months tight snug inch fine bit
6  SENTENCE: it is a great bag
7  BOW: flap logic stitching asus toshiba acer zip inches usb memory
7  SENTENCE: it is a great bag
8  BOW: carry perfect nice ipad handle work size carrying charger extra
8  SENTENCE: it is a great bag
9  BOW: pockets r

# confirm variables

In [27]:
_prob_topic, _prob_topic_sents, _prob_topic_infer, _means_topic_infer = debug_value([prob_topic, prob_topic_sents, prob_topic_infer, means_topic_infer], return_value=True)

In [28]:
batch_i = 4
_prob_topic_sents[batch_i], _prob_topic_infer[batch_i]

(array([0.07953417, 0.03046988, 0.08800525, 0.08928147, 0.21003477,
        0.12841846, 0.08533092, 0.05192988, 0.10679288, 0.13020231],
       dtype=float32),
 array([0.03983815, 0.03048858, 0.05354526, 0.09184394, 0.1779047 ,
        0.14580864, 0.08479333, 0.0659744 , 0.11976699, 0.19003609],
       dtype=float32))

In [29]:
_means_topic_infer[0][:, :4]

array([[-0.2152851 , -0.7954707 , -0.13175818,  0.5613478 ],
       [-0.17747657, -0.768313  , -0.11121741,  0.5308447 ],
       [-0.18701425, -0.7767259 , -0.11736107,  0.542892  ],
       [-0.17468357, -0.7817751 , -0.11434852,  0.53876525],
       [-0.18690863, -0.7800032 , -0.11768776,  0.5405479 ],
       [-0.17249909, -0.7702862 , -0.10337244,  0.53890795],
       [-0.184178  , -0.78831995, -0.11584231,  0.53070223],
       [-0.16777574, -0.76620877, -0.10552885,  0.5318073 ],
       [-0.16172384, -0.77200407, -0.11794638,  0.5309891 ],
       [-0.16318968, -0.77438337, -0.10628442,  0.5325533 ]],
      dtype=float32)

In [30]:
w_means_topic, b_means_topic = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "topic/dec/mean_topic")

pred_topic_embeddings, pred_topic_bow, pred_means_topic, pred_logvars_topic, pred_token_idxs, _w_means_topic, _b_means_topic, _w_mean_topic_infer = \
                                sess.run([topic_embeddings, topic_bow, means_topic, logvars_topic, topic_beam_output_token_idxs, w_means_topic, b_means_topic, w_mean_topic_infer], 
                                         feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})

pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)

pred_topics_freq_bow_indices = np.argsort(pred_topic_bow, 1)[:, ::-1][:, :10]
pred_topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]

In [31]:
for idxs in pred_topics_freq_bow_idxs:
    print([idx_to_word[idx] for idx in idxs])

['!', 'love', 'color', '...', 'recommend', 'perfect', 'perfectly', 'cute', 'ordered', 'buy']
['plug', 'vaio', 'rip', 'resistant', 'searching', 'squeeze', 'classy', 'functionality', 'local', 'abuse']
['cover', 'color', 'keyboard', 'bottom', 'mac', 'pro', 'top', 'hard', 'apple', 'scratches']
['broke', 'neoprene', 'thin', 'loose', 'foam', 'chromebook', 'started', 'month', 'close', 'returned']
[';', 'bought', '&', 'price', 'quality', 'nice', "'m", 'time', 'buy', 'perfectly']
['inside', 'made', '-', 'back', 'zipper', 'big', 'put', 'inch', "'ve", 'padding']
['sleeve', 'protection', 'material', 'zipper', 'months', 'tight', 'snug', 'fine', 'inch', ';']
['flap', 'asus', 'logic', 'stitching', 'toshiba', 'acer', 'zip', 'inches', 'usb', 'memory']
['carry', 'perfect', 'nice', 'ipad', 'work', 'handle', 'size', 'charger', 'carrying', 'extra']
['pockets', 'room', 'pocket', 'strap', 'shoulder', 'small', 'compartment', 'straps', 'plenty', 'items']


In [32]:
pred_topic_embeddings[:, :10]

array([[ 0.07030167,  0.34280053,  0.5620051 , -0.36249042, -0.37130642,
        -0.35529038, -0.52851105,  0.2897714 , -0.09859769,  0.02688825],
       [ 0.00454117, -0.14383823, -0.4358602 , -0.3000859 ,  0.36200735,
         0.6090798 ,  0.04143322,  0.25105503,  0.39387962, -0.2521794 ],
       [ 0.51439863, -0.5203031 ,  0.26585394,  0.18821827,  0.04780925,
        -0.2502492 , -0.31570742, -0.09478665, -0.15000348,  0.186701  ],
       [ 0.5495507 , -0.37986752,  0.37048703, -0.2601502 , -0.09304344,
         0.15158871, -0.40401134,  0.15019475,  0.20674033,  0.04148453],
       [ 0.5675219 ,  0.3590145 ,  0.06957109, -0.36036927, -0.4935817 ,
        -0.5302313 , -0.23160999, -0.39796302, -0.527676  ,  0.41196147],
       [-0.03068164,  0.42594528, -0.0811015 ,  0.53738195, -0.53529024,
        -0.43130898,  0.26999566,  0.04198602, -0.488762  ,  0.5043671 ],
       [ 0.46481928,  0.03554478,  0.32813412, -0.23864001, -0.30504122,
        -0.17199972,  0.2143851 , -0.01828023

In [33]:
pred_topic_bow

array([[2.70199955e-01, 3.15083051e-03, 5.96725731e-04, ...,
        5.76106868e-06, 7.55719593e-05, 2.34827962e-06],
       [3.09307943e-05, 8.20664536e-06, 1.75214736e-04, ...,
        1.94706372e-04, 1.93003834e-05, 3.84568446e-03],
       [2.76639767e-04, 1.80042302e-03, 3.83189268e-04, ...,
        1.13867945e-05, 3.29279865e-05, 5.01591921e-06],
       ...,
       [2.29592001e-06, 5.66102390e-05, 4.39598400e-04, ...,
        1.38007279e-04, 2.17028172e-03, 4.33204434e-04],
       [7.07316940e-05, 4.76617366e-04, 1.21361249e-04, ...,
        1.57259579e-04, 9.18528438e-03, 2.12362811e-05],
       [6.60414298e-06, 6.34977114e-05, 7.13269255e-05, ...,
        3.65038170e-03, 1.73785468e-03, 5.41338581e-04]], dtype=float32)

In [34]:
_w_means_topic

array([[-0.03424398,  0.0096985 , -0.06413093, ...,  0.03797114,
        -0.01338133, -0.01297894],
       [-0.03279095,  0.05930585,  0.03451761, ..., -0.02780341,
        -0.10020281,  0.03512232],
       [ 0.03666572,  0.08548579, -0.00936586, ..., -0.06135559,
        -0.09365493,  0.00985266],
       ...,
       [-0.04459329, -0.00699388, -0.007137  , ...,  0.10423539,
        -0.00303482,  0.028828  ],
       [-0.02674356, -0.01834767,  0.04752089, ...,  0.05400897,
        -0.02181414, -0.06615067],
       [ 0.07697956,  0.05168388,  0.07922779, ...,  0.09811964,
        -0.01671151, -0.01895013]], dtype=float32)

In [35]:
_b_means_topic

array([-0.01163579, -0.31689972, -0.06394058,  0.19231673, -0.1380821 ,
        0.24288706,  0.2933419 , -0.03508246,  0.08912257, -0.26633072,
       -0.37472242, -0.06411445, -0.38825554,  0.02678456, -0.05968409,
       -0.20403576, -0.02953517,  0.0643271 , -0.09376531, -0.00238578,
       -0.14848402,  0.39407536, -0.13403031,  0.24029757, -0.17921838,
       -0.01551062, -0.1424737 ,  0.23313083,  0.19225577, -0.30093712,
       -0.11009891, -0.01441393], dtype=float32)

In [36]:
pred_means_topic

array([[-0.20777631, -0.7880086 , -0.12567243,  0.56142694, -0.327168  ,
         0.60216874,  0.7404998 , -0.23602808,  0.289317  , -0.58346194,
        -1.0768209 , -0.08863895, -1.237956  ,  0.13280813, -0.09187369,
        -0.33609003, -0.1595888 ,  0.15013331, -0.20481265, -0.01800749,
        -0.4365478 ,  1.2110704 , -0.45148912,  0.688657  , -0.3711225 ,
        -0.06612568, -0.353558  ,  0.5758501 ,  0.44075334, -0.8059681 ,
        -0.25110504, -0.09649536],
       [-0.17156062, -0.7611089 , -0.11639434,  0.5273537 , -0.30693853,
         0.57691735,  0.6997352 , -0.2109155 ,  0.2612798 , -0.53453314,
        -1.0336132 , -0.07709154, -1.1726186 ,  0.10545208, -0.086261  ,
        -0.33172426, -0.13553052,  0.16665927, -0.16889113, -0.00922497,
        -0.39735556,  1.1698847 , -0.4418127 ,  0.66840285, -0.36015117,
        -0.07463156, -0.35274646,  0.5593424 ,  0.4279335 , -0.77467144,
        -0.2585973 , -0.07788712],
       [-0.18116352, -0.76906085, -0.11618015,  0.5401

In [37]:
_w_mean_topic_infer[:, :10, 0]

array([[-1.49821369e-02, -1.74026974e-02, -1.21583277e-03,
         6.53449260e-03, -1.54133858e-02, -2.18948219e-02,
        -1.90638546e-02,  1.32693239e-02,  1.11634033e-02,
        -1.93855334e-02],
       [-7.75512168e-03, -8.57588742e-03,  1.91637203e-02,
         6.88495440e-03,  5.73747957e-06, -1.56788658e-02,
         2.55801971e-03,  6.88684639e-03,  2.42787339e-02,
         1.79753304e-02],
       [ 4.94250655e-03, -1.07696308e-02, -1.53253425e-03,
         1.99725777e-02, -1.92840025e-02, -1.49392604e-03,
         1.78743172e-02,  7.22448155e-03,  1.98036060e-02,
         1.24130938e-02],
       [ 1.22360634e-02,  1.82047524e-02,  2.89237418e-04,
        -4.22067009e-03, -2.82733538e-03, -4.09162091e-03,
        -1.15149086e-02,  2.29803356e-03, -7.73391640e-03,
        -1.84050482e-02],
       [ 1.64312497e-02,  7.79688545e-03, -2.61567952e-03,
        -1.22013697e-02,  9.48593393e-03,  3.44272843e-03,
        -1.87890939e-02, -1.10813207e-03,  1.73099767e-02,
        -1.

In [38]:
_b_means_topic

array([-0.01163579, -0.31689972, -0.06394058,  0.19231673, -0.1380821 ,
        0.24288706,  0.2933419 , -0.03508246,  0.08912257, -0.26633072,
       -0.37472242, -0.06411445, -0.38825554,  0.02678456, -0.05968409,
       -0.20403576, -0.02953517,  0.0643271 , -0.09376531, -0.00238578,
       -0.14848402,  0.39407536, -0.13403031,  0.24029757, -0.17921838,
       -0.01551062, -0.1424737 ,  0.23313083,  0.19225577, -0.30093712,
       -0.11009891, -0.01441393], dtype=float32)

In [39]:
_enc_state_infer, _means_topic_infer = debug_value([enc_state_infer, means_topic_infer], return_value=True)

NameError: name 'enc_state_infer' is not defined

In [ ]:
_enc_state_infer.shape

In [ ]:
_means_topic_infer[0]